In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzbuilder_analysis.rendering as rg
import gzbuilder_analysis.fitting as fg
import gzbuilder_analysis.parsing as pg
import gzbuilder_analysis.config as cfg
try:
    from gzbuilder_analysis.rendering.cuda import calculate_model, oversampled_sersic_component
except ModuleNotFoundError:
    from gzbuilder_analysis.rendering import calculate_model, oversampled_sersic_component
from gzbuilder_analysis.fitting.model import Model, fit_model
from PIL import Image
import argparse

In [3]:
subject_id = 20902007

In [4]:
im = np.array(Image.open('lib/subject_data/{}/image.png'.format(subject_id)))[::-1]
classifications = pd.read_csv('lib/galaxy-builder-classifications.csv', index_col=0)
fm = pd.read_pickle('lib/fitting_metadata.pkl').loc[subject_id]
data = fm['galaxy_data']
sigma = fm['sigma_image']
psf = fm['psf']
gal = pd.read_csv('lib/gal-metadata.csv', index_col=0).loc[subject_id]
c = classifications.query('subject_ids == {}'.format(subject_id))

In [5]:
zoo_models = c.apply(
    pg.parse_classification,
    axis=1,
    # parse_classification kwargs
    image_size=np.array(im.shape),
    size_diff=im.shape[0] / data.shape[0],
)

scaled_models = zoo_models.apply(
    pg.scale_model,
    args=(fm['size_diff'],),
)

models = scaled_models.apply(
    pg.reproject_model,
    wcs_in=fm['montage_wcs'], wcs_out=fm['original_wcs']
)

In [6]:
print('Rendering models')
t0 = time()
rendered = models.apply(
    calculate_model,
    image_size=data.shape,
    psf=psf,
    oversample_n=3
)
print('\tElapsed time: {:.2f}'.format(time() - t0))
print('Calucating losses')
t0 = time()
chisq = rendered.apply(fg.chisq, args=(data, sigma))
print('\tElapsed time: {:.2f}'.format(time() - t0))

Rendering models
	Elapsed time: 17.84
Calucating losses
	Elapsed time: 0.12


In [7]:
idx_best = chisq.idxmin()

In [51]:
best_model = Model(
    models.loc[idx_best],
    data,
    psf=psf,
    sigma_image=sigma
)
np.allclose(
    rg.calculate_model(models.loc[chisq.idxmin()], data.shape, psf),
    best_model.render()
)

True

In [52]:
fit_model(best_model)

chisq=2.5187: : 21it [00:37,  1.52s/it]


Optimizing 7 parameters
Original chisq: 7.0361

Final chisq: 2.5187


(      fun: 2.5187039339129247
  hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
       jac: array([2.46425103e-04, 1.24344979e-06, 2.92654789e-05, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
      nfev: 200
       nit: 21
    status: 0
   success: True
         x: array([1.16230970e-01, 2.67802297e+01, 5.76192729e-01, 3.06250000e-01,
        1.15229348e-06, 5.00057342e-01, 5.00000000e+00]),
 {'disk': {'mux': 87.56684051257048,
   'muy': 86.88383947125118,
   'roll': -2.8980804208499067,
   'Re': 26.780229745703835,
   'q': 0.5761927294857382,
   'I': 0.11623096979859349,
   'n': 1.0,
   'c': 2.0},
  'bulge': {'mux': 87.9127704084915,
   'muy': 85.84679346085875,
   'roll': 0.4263741818919012,
   'Re': 1.152293478070874e-06,
   'q': 0.5000573419008862,
   'I': 0.30624999999999997,
   'n': 5.0,
   'c': 2.0},
  'bar': None,
  'spiral': []})